In [25]:
import pandas as pd
import plotly.express as px
import statistics as stats
import numpy as np
from datetime import datetime
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
import warnings
import itertools

In [30]:
smart_meter_data = pd.read_csv('value.csv', usecols=[' balance','date','lastTopup'])
# smart_meter_data =  pd.read_excel('log.xlsx',usecols=['balance','date','lastTopupAmount'])

In [18]:
# smart_meter_data['date'] = pd.to_datetime(smart_meter_data['date'], format='%Y-%m-%d')
# for i in range(len(smart_meter_data)):
#     smart_meter_data['date'][i].strftime("%Y-%m-%d")

# or do not convert atall because it would be in %Y-%m-%d format

In [27]:
fig = px.line(smart_meter_data[-31:-1], x='date', y='balance',title="Graph of user's consumption against time for last month")
fig.show()

ValueError: Value of 'y' is not the name of a column in 'data_frame'. Expected one of ['lastTopup', ' balance', ' lastTopupDate', ' weekConsumption', ' highestConsumptionDay', ' maximumConsumption', ' lowestConsumptionDay', ' minimumConsumption', ' averageConsumption', 'date'] but received: balance

In [5]:
fig = px.line(smart_meter_data[-7:], x='date', y='balance',title="Graph of user's consumption against time for last week")
fig.show()

# Calculation of average daily consumption

In [7]:
# balance = smart_meter_data['balance']
# mean_balance = balance.diff()
# mean_balance = abs(mean_balance)
# for i in range(len(mean_balance)):
#     if mean_balance[i] > 10:
#         mean_balance[i] = 1
# mean_balance = mean_balance.dropna()
# mean_balance = stats.mean(mean_balance)

# print('Your average daily consumption is GH₵%s'%round(mean_balance,2))

In [8]:
# print('Your average weekly consumption is GH₵%s'%(round(7 * mean_balance,2)))

# Prediction

In [28]:
smart_meter_data['Consumption'] = abs(smart_meter_data['balance'].diff()).dropna()
smart_meter_data['Consumption'] = smart_meter_data['Consumption'].replace(np.nan, 0)

KeyError: 'balance'

In [7]:
for i in range(len(smart_meter_data['Consumption'])):
    if smart_meter_data['Consumption'][i] >= 6:
#         smart_meter_data['Consumption'] = smart_meter_data['Consumption'].replace(smart_meter_data['Consumption'][i], 5)
        mean_consumption = stats.mean(smart_meter_data['Consumption'][:i])
        if mean_consumption <= 6:
            smart_meter_data['Consumption'] = smart_meter_data['Consumption'].replace(smart_meter_data['Consumption'][i], mean_consumption)
        else:
            smart_meter_data['Consumption'] = smart_meter_data['Consumption'].replace(smart_meter_data['Consumption'][i], 5)
        
# smart_meter_data['Consumption'].head(50)

In [8]:
smart_meter_data['Consumption'] = smart_meter_data['Consumption'].dropna()
prediction = smart_meter_data[['Consumption','date']]
prediction.index = pd.to_datetime(prediction.date)

In [9]:
ADF_result = adfuller(prediction['Consumption'].dropna())
while ADF_result[1] >= 0.05:
        d = 0

        abs(prediction['Consumption'].diff()).dropna()

        d = d+1

else:
        d=0
x = prediction['Consumption'].dropna().values
train = x[0:]
predictions = []

In [14]:
warnings.filterwarnings('ignore')
p=d=q = range(0,3)
pdq = list(itertools.product(p,d,q))
param_values = []
error_values = []
for param in pdq:
    try: 
        model_arima = ARIMA(train, order=param)
        model_arima_fit = model_arima.fit()
        print(param,model_arima_fit.aic)
        param_values.append(param)
        error_values.append(model_arima_fit.aic)
    except:
        continue

min_error_value = min(error_values)
error_value_index = error_values.index(min_error_value)
param_value = param_values[error_value_index]
print(error_values)

(0, 0, 0) 275.5884673622215
(0, 0, 1) 268.82150143468726
(0, 0, 2) 251.97028844814847
(0, 1, 0) 358.47182252171785
(0, 1, 1) 279.0815639644037
(0, 1, 2) 276.8811545660975
(0, 2, 0) 460.5786089217689
(0, 2, 1) 361.5033239543839
(1, 0, 0) 260.51757342946985
(1, 0, 1) 248.88568798787358
(1, 0, 2) 248.42033074851327
(1, 1, 0) 259.9542348587828
(1, 1, 1) 258.0439965469602
(1, 1, 2) 252.38075596978203
(1, 2, 0) 322.39756295208576
(1, 2, 1) 265.24428213434595
(1, 2, 2) 263.9144630767452
(2, 0, 0) 244.82771474279303
(2, 0, 1) 246.59912660537717
(2, 0, 2) 248.52996399420746
(2, 1, 0) 259.556254962082
(2, 1, 1) 248.49955922503727
(2, 1, 2) 250.29669987536315
(2, 2, 0) 303.13950172552853
(2, 2, 1) 265.1193633821458
[275.5884673622215, 268.82150143468726, 251.97028844814847, 358.47182252171785, 279.0815639644037, 276.8811545660975, 460.5786089217689, 361.5033239543839, 260.51757342946985, 248.88568798787358, 248.42033074851327, 259.9542348587828, 258.0439965469602, 252.38075596978203, 322.39756295

In [8]:
model_arima = ARIMA(train, order=param_value)
model_arima_fit = model_arima.fit()
forecast_values = list(model_arima_fit.forecast(steps=8)[0])
print(forecast_values)
min_forecast_value = min(forecast_values)
max_forecast_value = max(forecast_values)

[1.9717958276945626, 1.616161654820556, 1.96155108633747, 1.7231536057902268, 1.9319556862884486, 1.7774702081027733, 1.9061270408339839, 1.8075754661535295]


In [9]:
print('Your average daily consumption is between GH₵%s and GH₵%s'% (round(min_forecast_value,2), round(max_forecast_value,2)))
print('Your average weekly consumption is GH₵%s and GH₵%s'%(round(7 * min_forecast_value,2),round(7 * max_forecast_value,2)))
print('Your average monthly consumption is GH₵%s and GH₵%s'%(round(30 * min_forecast_value,2),round(30 * max_forecast_value,2)))

Your average daily consumption is between GH₵1.62 and GH₵1.97
Your average weekly consumption is GH₵11.31 and GH₵13.8
Your average monthly consumption is GH₵48.48 and GH₵59.15
